# sensR vs sensPy Validation Notebook

This notebook calls both **sensR** (R) and **sensPy** (Python) side-by-side to validate numerical parity.

### Requirements
- R with `sensR` package installed (`install.packages("sensR")`)
- Python: `rpy2`, `senspy`, `numpy`, `pandas`

```bash
pip install rpy2 pandas
```

In [ ]:
import numpy as np
import pandas as pd
from dataclasses import asdict

# rpy2 for calling R
import rpy2.robjects as ro
from rpy2.robjects import r, FloatVector, IntVector, StrVector
from rpy2.robjects.packages import importr

# sensPy
import senspy
from senspy import (
    discrim, betabin, twoac, samediff, anota, dod, dod_fit,
    discrim_power, dprime_power, discrim_sample_size, dprime_sample_size,
    dprime_test, dprime_compare, dprime_table,
    rescale, psy_fun, psy_inv, psy_deriv,
    sdt, roc, auc,
    pc_to_pd, pd_to_pc,
)

# Load sensR
sensR = importr('sensR')
base = importr('base')
stats = importr('stats')

# Helper to compare results
def compare(label, py_val, r_val, tol=1e-4):
    py_val = float(py_val)
    r_val = float(r_val)
    match = abs(py_val - r_val) < tol
    status = 'PASS' if match else 'FAIL'
    print(f"  {status} | {label:30s} | sensPy={py_val:12.6f} | sensR={r_val:12.6f} | diff={abs(py_val-r_val):.2e}")
    return match

print("Setup complete.")

---
## 1. `discrim()` — Discrimination Analysis

Test across multiple protocols and statistics.

In [ ]:
print("=" * 95)
print("1. discrim() — Basic discrimination analysis")
print("=" * 95)

test_cases = [
    {"correct": 80, "total": 100, "method": "triangle", "statistic": "exact"},
    {"correct": 60, "total": 100, "method": "duotrio", "statistic": "exact"},
    {"correct": 75, "total": 100, "method": "twoafc", "statistic": "likelihood"},
    {"correct": 50, "total": 100, "method": "threeafc", "statistic": "wald"},
    {"correct": 45, "total": 100, "method": "tetrad", "statistic": "score"},
    {"correct": 30, "total": 100, "method": "twoafc", "statistic": "exact"},
    {"correct": 90, "total": 120, "method": "triangle", "statistic": "likelihood"},
]

for tc in test_cases:
    print(f"\n--- correct={tc['correct']}, total={tc['total']}, method={tc['method']}, stat={tc['statistic']} ---")
    
    # Python
    py = discrim(tc["correct"], tc["total"], method=tc["method"], statistic=tc["statistic"])
    
    # R
    r_res = sensR.discrim(tc["correct"], tc["total"], method=tc["method"], statistic=tc["statistic"])
    r_coef = stats.coef(r_res)
    
    compare("d_prime", py.d_prime, r_coef.rx2("d.prime")[0])
    compare("pc", py.pc, r_coef.rx2("pc")[0])
    compare("pd", py.pd, r_coef.rx2("pd")[0])
    compare("se_d_prime", py.se_d_prime, r_res.rx2("coefficients")[1])
    compare("p_value", py.p_value, r_res.rx2("p.value")[0])

---
## 2. `discrim()` — Confidence Intervals

In [ ]:
print("=" * 95)
print("2. discrim() — Confidence Intervals")
print("=" * 95)

ci_cases = [
    {"correct": 80, "total": 100, "method": "triangle", "statistic": "exact"},
    {"correct": 60, "total": 100, "method": "twoafc", "statistic": "likelihood"},
    {"correct": 70, "total": 100, "method": "duotrio", "statistic": "wald"},
]

for tc in ci_cases:
    print(f"\n--- {tc['method']}, stat={tc['statistic']}, correct={tc['correct']}/{tc['total']} ---")
    
    py = discrim(tc["correct"], tc["total"], method=tc["method"], statistic=tc["statistic"])
    r_res = sensR.discrim(tc["correct"], tc["total"], method=tc["method"], statistic=tc["statistic"])
    
    for param in ["d_prime", "pc", "pd"]:
        py_ci = py.confint(parameter=param)
        r_param = {"d_prime": "d.prime", "pc": "pc", "pd": "pd"}[param]
        r_ci = stats.confint(r_res, **{"parm": r_param})
        compare(f"CI lower ({param})", py_ci[0], r_ci[0])
        compare(f"CI upper ({param})", py_ci[1], r_ci[1])

---
## 3. `discrim()` — Similarity Tests

In [ ]:
print("=" * 95)
print("3. discrim() — Similarity Tests (d_prime0 / pd0)")
print("=" * 95)

sim_cases = [
    {"correct": 40, "total": 100, "method": "triangle", "d_prime0": 1.0, "statistic": "exact"},
    {"correct": 55, "total": 100, "method": "twoafc", "d_prime0": 0.5, "statistic": "likelihood"},
]

for tc in sim_cases:
    print(f"\n--- {tc['method']}, d_prime0={tc['d_prime0']}, stat={tc['statistic']} ---")
    
    py = discrim(tc["correct"], tc["total"], method=tc["method"],
                 d_prime0=tc["d_prime0"], statistic=tc["statistic"], test="similarity")
    r_res = sensR.discrim(tc["correct"], tc["total"], method=tc["method"],
                          **{"d.prime0": tc["d_prime0"]}, statistic=tc["statistic"], test="similarity")
    
    r_coef = stats.coef(r_res)
    compare("d_prime", py.d_prime, r_coef.rx2("d.prime")[0])
    compare("p_value", py.p_value, r_res.rx2("p.value")[0])

---
## 4. `rescale()` — Scale Conversions

In [ ]:
print("=" * 95)
print("4. rescale() — Convert between pc, pd, d_prime")
print("=" * 95)

protocols = ["triangle", "twoafc", "duotrio", "threeafc", "tetrad"]

# From d_prime
for method in protocols:
    for dp in [0.5, 1.0, 2.0, 3.0]:
        py_res = rescale(d_prime=dp, method=method)
        r_res = sensR.rescale(FloatVector([dp]), method=method)
        r_coef = stats.coef(r_res)
        
        ok1 = compare(f"{method} d'={dp} -> pc", py_res.pc, r_coef.rx2("pc")[0])
        ok2 = compare(f"{method} d'={dp} -> pd", py_res.pd, r_coef.rx2("pd")[0])

print("\n--- From pc ---")
for method in ["triangle", "twoafc"]:
    for pc_val in [0.5, 0.7, 0.9]:
        py_res = rescale(pc=pc_val, method=method)
        r_res = sensR.rescale(pc=FloatVector([pc_val]), method=method)
        r_coef = stats.coef(r_res)
        compare(f"{method} pc={pc_val} -> d'", py_res.d_prime, r_coef.rx2("d.prime")[0])

print("\n--- From pd ---")
for method in ["triangle", "twoafc"]:
    for pd_val in [0.2, 0.5, 0.8]:
        py_res = rescale(pd=pd_val, method=method)
        r_res = sensR.rescale(pd=FloatVector([pd_val]), method=method)
        r_coef = stats.coef(r_res)
        compare(f"{method} pd={pd_val} -> d'", py_res.d_prime, r_coef.rx2("d.prime")[0])

---
## 5. `psy_fun()` / `psy_inv()` / `psy_deriv()` — Psychometric Links

In [ ]:
print("=" * 95)
print("5. psyfun / psyinv / psyderiv — Psychometric link functions")
print("=" * 95)

d_primes = [0.0, 0.5, 1.0, 1.5, 2.0, 3.0]
protocols = ["triangle", "twoafc", "duotrio", "threeafc", "tetrad"]

for method in protocols:
    print(f"\n--- {method} ---")
    for dp in d_primes:
        # psyfun: d' -> pc
        py_pc = float(psy_fun(dp, method=method))
        r_pc = float(sensR.psyfun(dp, method=method)[0])
        compare(f"psyfun d'={dp}", py_pc, r_pc)
    
    # psyinv: pc -> d'
    pc_vals = [0.5, 0.7, 0.9]
    for pc_val in pc_vals:
        py_dp = float(psy_inv(pc_val, method=method))
        r_dp = float(sensR.psyinv(pc_val, method=method)[0])
        compare(f"psyinv pc={pc_val}", py_dp, r_dp)
    
    # psyderiv
    for dp in [0.5, 1.0, 2.0]:
        py_deriv = float(psy_deriv(dp, method=method))
        r_deriv = float(sensR.psyderiv(dp, method=method)[0])
        compare(f"psyderiv d'={dp}", py_deriv, r_deriv)

---
## 6. `betabin()` — Beta-Binomial Model

In [ ]:
print("=" * 95)
print("6. betabin() — Beta-Binomial overdispersion model")
print("=" * 95)

# Panel data: (correct, total) per panelist
data = np.array([
    [3, 10], [5, 10], [7, 10], [4, 10], [6, 10],
    [8, 10], [2, 10], [5, 10], [9, 10], [4, 10],
])

for corrected in [True, False]:
    for method in ["duotrio", "triangle", "twoafc"]:
        print(f"\n--- method={method}, corrected={corrected} ---")
        
        # Python
        py = betabin(data, method=method, corrected=corrected)
        
        # R
        r_data = ro.r.matrix(IntVector(data.flatten()), nrow=data.shape[0], ncol=2, byrow=True)
        r_res = sensR.betabin(r_data, method=method, corrected=corrected)
        r_coef = stats.coef(r_res)
        
        compare("mu", py.mu, r_coef[0])
        compare("gamma", py.gamma, r_coef[1])
        compare("log_likelihood", py.log_likelihood, float(r_res.rx2("logLik")[0]))

---
## 7. `twoac()` — 2-AC Protocol

In [ ]:
print("=" * 95)
print("7. twoac() — 2-Alternative Constant protocol")
print("=" * 95)

twoac_cases = [
    {"data": [40, 20, 60], "statistic": "likelihood"},
    {"data": [30, 40, 50], "statistic": "wald"},
    {"data": [10, 80, 30], "statistic": "likelihood"},
    {"data": [55, 10, 35], "statistic": "likelihood"},
]

for tc in twoac_cases:
    print(f"\n--- data={tc['data']}, stat={tc['statistic']} ---")
    
    # Python
    py = twoac(tc["data"], statistic=tc["statistic"])
    
    # R
    r_res = sensR.twoAC(IntVector(tc["data"]), statistic=tc["statistic"])
    r_coef = r_res.rx2("coefficients")
    
    compare("tau", py.tau, r_coef[0])      # [0,0]
    compare("d_prime", py.d_prime, r_coef[1])  # [1,0]
    compare("se_tau", py.se_tau, r_coef[2])    # [0,1]
    compare("se_d_prime", py.se_d_prime, r_coef[3])  # [1,1]
    compare("log_likelihood", py.log_likelihood, float(r_res.rx2("logLik")[0]))
    if py.p_value is not None:
        compare("p_value", py.p_value, float(r_res.rx2("p.value")[0]))

---
## 8. `samediff()` — Same-Different Protocol

In [ ]:
print("=" * 95)
print("8. samediff() — Same-Different protocol")
print("=" * 95)

sd_cases = [
    {"ss": 45, "ds": 5, "sd": 20, "dd": 30},
    {"ss": 80, "ds": 20, "sd": 10, "dd": 90},
    {"ss": 30, "ds": 10, "sd": 15, "dd": 25},
]

for tc in sd_cases:
    print(f"\n--- ss={tc['ss']}, ds={tc['ds']}, sd={tc['sd']}, dd={tc['dd']} ---")
    
    # Python
    py = samediff(nsamesame=tc["ss"], ndiffsame=tc["ds"],
                  nsamediff=tc["sd"], ndiffdiff=tc["dd"])
    
    # R
    r_res = sensR.samediff(nsamesame=tc["ss"], ndiffsame=tc["ds"],
                           nsamediff=tc["sd"], ndiffdiff=tc["dd"])
    r_coef = r_res.rx2("coefficients")
    
    compare("tau", py.tau, r_coef[0])
    compare("delta (d')", py.delta, r_coef[1])
    compare("se_tau", py.se_tau, r_coef[2])
    compare("se_delta", py.se_delta, r_coef[3])
    compare("log_likelihood", py.log_likelihood, float(r_res.rx2("logLik")[0]))

---
## 9. `anota()` — A-Not-A Protocol

In [ ]:
print("=" * 95)
print("9. anota() — A-Not-A protocol")
print("=" * 95)

anota_cases = [
    {"x1": 80, "n1": 100, "x2": 90, "n2": 100},
    {"x1": 60, "n1": 80, "x2": 70, "n2": 80},
    {"x1": 40, "n1": 50, "x2": 45, "n2": 50},
]

for tc in anota_cases:
    print(f"\n--- x1={tc['x1']}, n1={tc['n1']}, x2={tc['x2']}, n2={tc['n2']} ---")
    
    # Python
    py = anota(tc["x1"], tc["n1"], tc["x2"], tc["n2"])
    
    # R
    r_res = sensR.AnotA(tc["x1"], tc["n1"], tc["x2"], tc["n2"])
    
    compare("d_prime", py.d_prime, float(r_res.rx2("dprime")[0]))
    compare("se_d_prime", py.se_d_prime, float(r_res.rx2("se")[0]))
    compare("p_value", py.p_value, float(r_res.rx2("p.value")[0]))
    compare("hit_rate", py.hit_rate, float(r_res.rx2("hit.rate")[0]))
    compare("false_alarm_rate", py.false_alarm_rate, float(r_res.rx2("fa.rate")[0]))

---
## 10. `dod()` — Degree of Difference Model

In [ ]:
print("=" * 95)
print("10. dod() — Degree of Difference model")
print("=" * 95)

# Example DOD data (4 response categories)
dod_cases = [
    {"same": [25, 22, 33, 20], "diff": [10, 15, 30, 45]},
    {"same": [40, 30, 20, 10], "diff": [5, 15, 30, 50]},
]

for tc in dod_cases:
    print(f"\n--- same={tc['same']}, diff={tc['diff']} ---")
    
    # Python
    py = dod(tc["same"], tc["diff"])
    
    # R
    r_res = sensR.dod(IntVector(tc["same"]), IntVector(tc["diff"]))
    
    compare("d_prime", py.d_prime, float(r_res.rx2("d.prime")[0]))
    compare("se_d_prime", py.se_d_prime, float(r_res.rx2("se")[0]))
    compare("log_likelihood", py.log_likelihood, float(r_res.rx2("logLik")[0]))
    compare("p_value", py.p_value, float(r_res.rx2("p.value")[0]))
    
    # Compare tau estimates
    r_tau = list(r_res.rx2("tau"))
    for i, (py_t, r_t) in enumerate(zip(py.tau, r_tau)):
        compare(f"tau[{i}]", py_t, r_t)

---
## 11. `dod_fit()` and `optimal_tau()`

In [ ]:
from senspy import optimal_tau, par2prob_dod

print("=" * 95)
print("11. dod_fit() and optimal_tau()")
print("=" * 95)

# optimal_tau
for dp in [0.5, 1.0, 2.0]:
    for ncat in [3, 4, 5]:
        py_tau = optimal_tau(dp, n_categories=ncat)
        r_tau = sensR.optimal_tau(dp, **{"n.cat": ncat})
        print(f"\n--- optimal_tau d'={dp}, n_cat={ncat} ---")
        for i in range(len(py_tau)):
            compare(f"tau[{i}]", py_tau[i], float(r_tau[i]))

# par2prob_dod
print("\n--- par2prob_dod ---")
tau = np.array([1.0, 2.0, 3.0])
dp = 1.5
py_prob = par2prob_dod(tau, dp)
r_prob = sensR.par2prob_dod(FloatVector(tau), dp)
r_prob_matrix = np.array(r_prob).reshape(2, -1)
for i in range(py_prob.shape[0]):
    for j in range(py_prob.shape[1]):
        label = f"prob[{['same','diff'][i]},{j}]"
        compare(label, py_prob[i, j], r_prob_matrix[i, j])

---
## 12. `discrim_power()` / `dprime_power()` — Power Analysis

In [ ]:
print("=" * 95)
print("12. Power analysis — discrim_power() / dprime_power()")
print("=" * 95)

# discrim_power
print("\n--- discrim_power ---")
power_cases = [
    {"pd_a": 0.5, "n": 100, "p_guess": 1/3, "stat": "exact"},
    {"pd_a": 0.3, "n": 50, "p_guess": 0.5, "stat": "exact"},
    {"pd_a": 0.4, "n": 80, "p_guess": 1/3, "stat": "normal"},
]

for tc in power_cases:
    py_pow = discrim_power(pd_a=tc["pd_a"], sample_size=tc["n"],
                           p_guess=tc["p_guess"], statistic=tc["stat"])
    r_pow = sensR.discrimPwr(tc["pd_a"], tc["n"],
                              **{"pGuess": tc["p_guess"]}, statistic=tc["stat"])
    compare(f"power pd_a={tc['pd_a']}, n={tc['n']}", py_pow, float(r_pow[0]))

# dprime_power
print("\n--- dprime_power ---")
dp_power_cases = [
    {"d_prime_a": 1.0, "n": 100, "method": "triangle", "stat": "exact"},
    {"d_prime_a": 1.5, "n": 50, "method": "twoafc", "stat": "exact"},
    {"d_prime_a": 2.0, "n": 80, "method": "duotrio", "stat": "exact"},
]

for tc in dp_power_cases:
    py_pow = dprime_power(d_prime_a=tc["d_prime_a"], sample_size=tc["n"],
                          method=tc["method"], statistic=tc["stat"])
    r_pow = sensR.discrimPwr(
        float(rescale(d_prime=tc["d_prime_a"], method=tc["method"]).pd),
        tc["n"],
        **{"pGuess": senspy.core.types.parse_protocol(tc["method"]).p_guess},
        statistic=tc["stat"]
    )
    compare(f"d'={tc['d_prime_a']}, {tc['method']}, n={tc['n']}", py_pow, float(r_pow[0]))

---
## 13. `discrim_sample_size()` / `dprime_sample_size()` — Sample Size

In [ ]:
print("=" * 95)
print("13. Sample size estimation")
print("=" * 95)

ss_cases = [
    {"pd_a": 0.5, "power": 0.8, "p_guess": 1/3, "stat": "exact"},
    {"pd_a": 0.3, "power": 0.9, "p_guess": 0.5, "stat": "exact"},
    {"pd_a": 0.4, "power": 0.8, "p_guess": 1/3, "stat": "normal"},
]

for tc in ss_cases:
    py_n = discrim_sample_size(pd_a=tc["pd_a"], power=tc["power"],
                               p_guess=tc["p_guess"], statistic=tc["stat"])
    r_n = sensR.discrimSS(tc["pd_a"], tc["power"],
                           **{"pGuess": tc["p_guess"]}, statistic=tc["stat"])
    compare(f"n: pd_a={tc['pd_a']}, power={tc['power']}", py_n, float(r_n[0]))

print("\n--- dprime_sample_size ---")
dp_ss_cases = [
    {"d_prime_a": 1.0, "power": 0.8, "method": "triangle"},
    {"d_prime_a": 1.5, "power": 0.9, "method": "twoafc"},
]

for tc in dp_ss_cases:
    py_n = dprime_sample_size(d_prime_a=tc["d_prime_a"], power=tc["power"],
                              method=tc["method"])
    print(f"  sensPy n={py_n} for d'={tc['d_prime_a']}, power={tc['power']}, {tc['method']}")
    print(f"  (Compare with R: sensR::findcr / manual calculation)")

---
## 14. `dprime_test()` — Common d-prime Test

In [ ]:
print("=" * 95)
print("14. dprime_test() — Test common d-prime")
print("=" * 95)

correct = [80, 65, 90]
total = [100, 100, 100]
protocol = ["triangle", "twoafc", "duotrio"]

for stat in ["likelihood", "wald"]:
    for alt in ["greater", "two.sided"]:
        print(f"\n--- statistic={stat}, alternative={alt} ---")
        
        # Python
        py = dprime_test(correct, total, protocol,
                         statistic=stat, alternative=alt)
        
        # R
        r_res = r(f'''
            sensR::dprime_test(
                correct = c({','.join(map(str, correct))}),
                total = c({','.join(map(str, total))}),
                protocol = c("{'","'.join(protocol)}"),
                statistic = "{stat}",
                alternative = "{alt}"
            )
        ''')
        
        compare("d_prime", py.d_prime, float(r_res.rx2("d.prime")[0]))
        compare("se_d_prime", py.se_d_prime, float(r_res.rx2("se")[0]))
        compare("stat_value", py.stat_value, float(r_res.rx2("stat.value")[0]))
        compare("p_value", py.p_value, float(r_res.rx2("p.value")[0]))

---
## 15. `dprime_compare()` — Compare d-primes Across Groups

In [ ]:
print("=" * 95)
print("15. dprime_compare() — Test equality of d-primes")
print("=" * 95)

correct = [80, 65, 90, 45]
total = [100, 100, 100, 100]
protocol = ["triangle", "triangle", "twoafc", "duotrio"]

for stat in ["likelihood", "wald"]:
    print(f"\n--- statistic={stat} ---")
    
    # Python
    py = dprime_compare(correct, total, protocol, statistic=stat)
    
    # R
    r_res = r(f'''
        sensR::dprime_compare(
            correct = c({','.join(map(str, correct))}),
            total = c({','.join(map(str, total))}),
            protocol = c("{'","'.join(protocol)}"),
            statistic = "{stat}"
        )
    ''')
    
    compare("d_prime (common)", py.d_prime, float(r_res.rx2("d.prime")[0]))
    compare("chi_sq statistic", py.stat_value, float(r_res.rx2("stat.value")[0]))
    compare("p_value", py.p_value, float(r_res.rx2("p.value")[0]))
    compare("df", py.df, float(r_res.rx2("df")[0]))

---
## 16. `dprime_table()` — Per-group d-prime Estimates

In [ ]:
print("=" * 95)
print("16. dprime_table() — Individual group estimates")
print("=" * 95)

correct = [80, 65, 90]
total = [100, 100, 100]
protocol = ["triangle", "twoafc", "duotrio"]

# Python
py_table = dprime_table(correct, total, protocol)

# R
r_table = r(f'''
    sensR::dprime_table(
        correct = c({','.join(map(str, correct))}),
        total = c({','.join(map(str, total))}),
        protocol = c("{'","'.join(protocol)}")
    )
''')

r_matrix = np.array(r_table)
for i, row in enumerate(py_table):
    print(f"\n--- Group {i+1}: {protocol[i]} ({correct[i]}/{total[i]}) ---")
    compare(f"d_prime", row.d_prime, r_matrix[i])
    compare(f"se", row.se, r_matrix[i + len(py_table)])

---
## 17. `sdt()` / `roc()` / `auc()` — Signal Detection Theory

In [ ]:
print("=" * 95)
print("17. sdt() / roc() / auc() — Signal Detection Theory")
print("=" * 95)

# Contingency table: 2 x J (signal/noise rows, J response categories)
table = np.array([
    [30, 40, 20, 10],   # signal (hits descending confidence)
    [5, 15, 30, 50],    # noise (false alarms descending confidence)
])

# SDT
print("\n--- sdt() ---")
py_sdt = sdt(table)
r_sdt = sensR.SDT(ro.r.matrix(IntVector(table.flatten()), nrow=2, byrow=True))
r_sdt_matrix = np.array(r_sdt)
n_criteria = py_sdt.shape[0]
for i in range(n_criteria):
    compare(f"d_prime[{i}]", py_sdt[i, 2], r_sdt_matrix[i + 2 * n_criteria])  # 3rd column

# AUC
print("\n--- auc() ---")
for dp in [0.5, 1.0, 2.0, 3.0]:
    py_auc = auc(dp)
    r_auc = sensR.AUC(dp)
    compare(f"AUC d'={dp}", py_auc.value, float(r_auc[0]))

print("\n--- auc() with SE and CI ---")
py_auc = auc(1.5, se_d=0.3)
r_auc = sensR.AUC(1.5, **{"se.d": 0.3})
compare("AUC value", py_auc.value, float(r_auc.rx2("AUC")[0]))
compare("AUC lower", py_auc.lower, float(r_auc.rx2("lower")[0]))
compare("AUC upper", py_auc.upper, float(r_auc.rx2("upper")[0]))

---
## 18. `pc_to_pd()` / `pd_to_pc()` — Simple Conversions

In [ ]:
print("=" * 95)
print("18. pc_to_pd() / pd_to_pc() — Direct conversions")
print("=" * 95)

p_guesses = [1/3, 0.5, 0.1]

for pg in p_guesses:
    print(f"\n--- p_guess = {pg:.4f} ---")
    for pc_val in [0.5, 0.7, 0.9, 1.0]:
        if pc_val >= pg:
            py_pd = float(pc_to_pd(pc_val, pg))
            r_pd = float(sensR.pc2pd(pc_val, pg)[0])
            compare(f"pc2pd pc={pc_val}", py_pd, r_pd)
    
    for pd_val in [0.0, 0.3, 0.5, 1.0]:
        py_pc = float(pd_to_pc(pd_val, pg))
        r_pc = float(sensR.pd2pc(pd_val, pg)[0])
        compare(f"pd2pc pd={pd_val}", py_pc, r_pc)

---
## 19. Comprehensive Cross-Protocol Comparison

A systematic test of `discrim()` across all protocols with a fixed dataset.

In [ ]:
print("=" * 95)
print("19. Cross-protocol comparison — discrim() for all protocols")
print("=" * 95)

all_protocols = ["triangle", "duotrio", "twoafc", "threeafc", "tetrad"]
# Use 70/100 as a common test case
correct, total = 70, 100

rows = []
for method in all_protocols:
    py = discrim(correct, total, method=method)
    r_res = sensR.discrim(correct, total, method=method)
    r_coef = stats.coef(r_res)
    
    rows.append({
        "protocol": method,
        "py_dprime": py.d_prime,
        "R_dprime": float(r_coef.rx2("d.prime")[0]),
        "py_pc": py.pc,
        "R_pc": float(r_coef.rx2("pc")[0]),
        "py_pd": py.pd,
        "R_pd": float(r_coef.rx2("pd")[0]),
        "py_pval": py.p_value,
        "R_pval": float(r_res.rx2("p.value")[0]),
    })

df = pd.DataFrame(rows)
df["dprime_diff"] = abs(df["py_dprime"] - df["R_dprime"])
df["pc_diff"] = abs(df["py_pc"] - df["R_pc"])
df["pd_diff"] = abs(df["py_pd"] - df["R_pd"])
df["pval_diff"] = abs(df["py_pval"] - df["R_pval"])
print(df.to_string(index=False))
print(f"\nMax d-prime diff: {df['dprime_diff'].max():.2e}")
print(f"Max p-value diff: {df['pval_diff'].max():.2e}")

---
## 20. Summary — Final Validation Report

In [ ]:
print("=" * 95)
print("VALIDATION SUMMARY")
print("=" * 95)
print("""
Functions validated:
  1.  discrim()           - discrimination analysis (all protocols & statistics)
  2.  discrim() CI        - confidence intervals (d_prime, pc, pd)
  3.  discrim() similarity- similarity tests with d_prime0
  4.  rescale()           - pc/pd/d_prime conversions
  5.  psyfun/psyinv/deriv - psychometric link functions
  6.  betabin()           - beta-binomial overdispersion model
  7.  twoac()             - 2-AC protocol analysis
  8.  samediff()          - same-different protocol
  9.  anota()             - A-Not-A protocol
  10. dod()               - degree of difference model
  11. dod_fit/optimal_tau - DOD utilities
  12. discrim_power()     - power analysis
  13. sample_size()       - sample size estimation
  14. dprime_test()       - common d-prime hypothesis test
  15. dprime_compare()    - d-prime equality test
  16. dprime_table()      - per-group d-prime table
  17. sdt/roc/auc         - signal detection theory
  18. pc2pd/pd2pc         - direct conversions
  19. Cross-protocol      - systematic all-protocol comparison

Review any FAIL lines above to investigate discrepancies.
""")